In [2]:

import open3d as o3d 
import plotly
import os.path as osp 
import os 
import numpy as np
import trimesh
%pwd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-05-10 14:38:42,077 - helper - Adding Anaconda lib path:/home/junting/miniforge3/envs/sslgrasp/lib/


'/home/junting/repo/ndf_robot/jupyter_notebooks'

In [3]:
object_paths = "../src/ndf_robot/descriptions/objects"


## Demo files 

In [4]:
# arguments and paths 
obj_class = "mug"
demo_exp = "grasp_rim_hang_handle_gaussian_precise_w_shelf"
demo_dir = "../src/ndf_robot/data/demos"
demo_load_dir = osp.join(demo_dir, obj_class, demo_exp)

In [5]:
# load demo files 
demo_filenames = os.listdir(demo_load_dir)
# strip the filenames to properly pair up each demo file
grasp_demo_filenames_orig = [
    osp.join(demo_load_dir,
                fn) for fn in demo_filenames if 'grasp_demo' in fn]  # use the grasp names as a reference

place_demo_filenames = []
grasp_demo_filenames = []
for i, fname in enumerate(grasp_demo_filenames_orig):
    shapenet_id_npz = fname.split('/')[-1].split('grasp_demo_')[-1]
    place_fname = osp.join(
        '/'.join(fname.split('/')[:-1]), 'place_demo_' + shapenet_id_npz)
    if osp.exists(place_fname):
        grasp_demo_filenames.append(fname)
        place_demo_filenames.append(place_fname)
    else:
        print(
            'Could not find corresponding placement demo: %s, skipping ' % place_fname)


In [6]:
# load 
file_idx = 0
grasp_demo_fn = grasp_demo_filenames[file_idx]
place_demo_fn = place_demo_filenames[file_idx]
grasp_data = np.load(grasp_demo_fn, allow_pickle=True)
place_data = np.load(place_demo_fn, allow_pickle=True)

In [7]:
print("list of keys: ", list(grasp_data.keys()))
print("grasp_data['shapenet_id']: ", grasp_data['shapenet_id'])

list of keys:  ['shapenet_id', 'ee_pose_world', 'robot_joints', 'obj_pose_world', 'obj_pose_camera', 'object_pointcloud', 'depth', 'seg', 'camera_poses', 'obj_model_file', 'obj_model_file_dec', 'gripper_pts', 'gripper_pts_gaussian', 'gripper_pts_uniform', 'gripper_contact_pose', 'table_urdf']
grasp_data['shapenet_id']:  b811555ccf5ef6c4948fa2daa427fe1f


In [15]:
grasp_data['shapenet_id']

array('b811555ccf5ef6c4948fa2daa427fe1f', dtype='<U32')

## Check object files in ShapeNet 

In [9]:
ndf_objects_path = "../src/ndf_robot/descriptions/objects"
shapenet_objects_path = "/home/junting/Downloads/datasets/shapenet_v2"

In [10]:
# make sure all objects in ndf are provided in shapenet
shapenet_id_dict = {'mug':'03797390', 'bottle': '02876657', 'bowl':'02880940'}
missing_objects = {'mug':[], 'bottle':[], 'bowl':[]}

for ndf_cls, sn_id, in shapenet_id_dict.items():
    ndf_cls_object_ids = os.listdir(osp.join(ndf_objects_path, f"{ndf_cls}_centered_obj_normalized"))
    sn_id_object_ids = os.listdir(osp.join(shapenet_objects_path, sn_id)) 
    
    for ndf_id in ndf_cls_object_ids:
        if ndf_id not in sn_id_object_ids:
            missing_objects[ndf_cls].append(ndf_id)

for ndf_cls in missing_objects.keys():
    print(f"Class {ndf_cls} missing objects: {missing_objects[ndf_cls]}")

# Output: 
# Class mug missing objects: []
# Class bottle missing objects: []
# Class bowl missing objects: []

Class mug missing objects: []
Class bottle missing objects: []
Class bowl missing objects: []


In [11]:
# compare difference between ndf objects and shapenet objects 
ndf_cls = 'mug'
object_idx = 20
sn_id = shapenet_id_dict[ndf_cls]

ndf_cls_object_ids = os.listdir(osp.join(ndf_objects_path, f"{ndf_cls}_centered_obj_normalized"))
sn_id_object_ids = os.listdir(osp.join(shapenet_objects_path, sn_id)) 

ndf_obj_file_path =  osp.join(
    ndf_objects_path, 
    f"{ndf_cls}_centered_obj_normalized", 
    ndf_cls_object_ids[object_idx],
    "models",
    "model_normalized.obj")
sn_obj_file_path =  osp.join(
    shapenet_objects_path, 
    sn_id, 
    ndf_cls_object_ids[object_idx],
    "models",
    "model_normalized.obj")
print(f"Object id: {ndf_cls_object_ids[object_idx]}")

Object id: 645b0e2ef3b95979204df312eabf367f


In [12]:
# trimesh.util.attach_to_log() 
ndf_obj_trimesh = trimesh.load(ndf_obj_file_path)
sn_obj_trimesh = trimesh.load(sn_obj_file_path)
ndf_obj_trimesh.apply_translation((-1, 0, 0))
sn_obj_trimesh.apply_translation((1, 0, 0))

# trimesh.Scene([ndf_obj_trimesh, sn_obj_trimesh]).show(viewer='gl')